In [1]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

In [2]:
df = pd.read_csv('./data/apache_sprint_issues.csv')

In [3]:
unique_projects = df['project'].unique()

In [7]:
df.head()

,boardId,sprintId,key,fixVersion,priority,status,status_category,creator,reporter,progress,progress_total,worklog,issuetype,project,created,updated,description,summary,duedate
0,37,7,AURORA-350,0.5.0,Major,Resolved,Done,Maxim Khutornenko,Maxim Khutornenko,0,0,0,Story,Aurora,2014-04-24T00:07:33.000+0000,2014-08-18T17:35:36.000+0000,The way aurora deploy works inherently contrib...,Parallelize updates to speed up deploys,NaN
1,37,7,AURORA-149,0.5.0,Major,Resolved,Done,Jake Farrell,Jake Farrell,0,0,0,Story,Aurora,2014-01-31T19:59:34.000+0000,2014-06-25T20:23:57.000+0000,NaN,python code should have a checkstyle run as pa...,NaN
2,37,7,AURORA-421,0.5.0,Major,Resolved,Done,Bill Farner,Bill Farner,0,0,0,Task,Aurora,2014-05-14T21:10:31.000+0000,2014-07-08T13:16:38.000+0000,NaN,H2-backed implementation of QuotaStore,NaN
3,37,7,AURORA-433,0.5.0,Major,Resolved,Done,Mark Chu-Carroll,Mark Chu-Carroll,0,0,0,Task,Aurora,2014-05-15T17:43:54.000+0000,2014-07-08T13:16:38.000+0000,NaN,Write a design doc describing the aurora short...,NaN
4,37,7,AURORA-335,0.5.0,Major,Resolved,Done,Bill Farner,Bill Farner,0,0,0,Task,Aurora,2014-04-21T20:07:54.000+0000,2014-06-25T20:23:58.000+0000,NaN,H2-backed implementation of LockStore,NaN


In [10]:
df['reporter']

0        Maxim Khutornenko
1             Jake Farrell
2              Bill Farner
3         Mark Chu-Carroll
4              Bill Farner
               ...        
68875               Alex R
68876      Benjamin Mahler
68877           Paul Brett
68878           Vinod Kone
68879               Adam B
Name: reporter, Length: 68880, dtype: object

In [14]:
employee_projects = df.groupby('project')['reporter'].unique().reset_index()

In [15]:
employee_projects.head()

,project,reporter
0,Apache DataLab (Retired),"[Vira Vitanska, Anna Orlovska, Demyan Mysakove..."
1,Apache Fineract,[Javier Borkenztain]
2,Apache Gobblin,"[Abhishek Tiwari, Michał Woś, Lei Sun, Arjun S..."
3,Apache Helix,"[kishore gopalakrishna, Kanak Biscuitwala, Zhe..."
4,Apache Hudi,"[sivabalan narayanan, 董可伦, tao meng, Manoj Gov..."


In [17]:
# Filter employees involved in multiple projects
employee_projects = employee_projects[employee_projects['project'].apply(len) > 1]
employee_projects

,project,reporter
0,Apache DataLab (Retired),"[Vira Vitanska, Anna Orlovska, Demyan Mysakove..."
1,Apache Fineract,[Javier Borkenztain]
2,Apache Gobblin,"[Abhishek Tiwari, Michał Woś, Lei Sun, Arjun S..."
3,Apache Helix,"[kishore gopalakrishna, Kanak Biscuitwala, Zhe..."
4,Apache Hudi,"[sivabalan narayanan, 董可伦, tao meng, Manoj Gov..."
5,Apache IoTDB,"[xiangdong Huang, Julian Feinauer, Yuan Tian, ..."
6,Apache MXNet (Retired),"[Rahul Huilgol, Chris Olivier, Haibin Lin, Lai..."
7,Apache Ozone,"[Nandakumar, Lokesh Jain, Xiaoyu Yao, Mukul Ku..."
8,Apache Science Data Analytics Platform,"[Eamon Ford, Thomas G Loubrieu]"
9,AriaTosca,"[Tal Liron, Thomas Nadeau, Maxim Orlov, Ran Zi..."


In [20]:
project_count = df.groupby('reporter')['project'].nunique().reset_index()

# Filter employees working on more than one project
multi_project_employees = project_count[project_count['project'] > 1]
multi_project_employees

,reporter,project
68,Benjamin Mahler,2
73,Bhuvaneswaran A,2
74,Bill Farner,2
76,Bjoern Metzdorf,2
97,Chris Lambert,2
133,Dominic Hamon,2
214,Jay Buffington,2
230,Joe Smith,2
286,Lokesh Jain,2
308,Maxim Khutornenko,2


In [23]:
employee_projects = df.groupby('reporter')['project'].unique().reset_index()

# Optionally, convert list of projects into a comma-separated string for easy reading
employee_projects['project'] = employee_projects['project']


employee_projects

,reporter,project
0,A. Dukhovniy,[Mesos]
1,Aaron Bell,[Mesos]
2,Aaron Markham,[Apache MXNet (Retired)]
3,Abhishek Tiwari,[Apache Gobblin]
4,Adam B,[Mesos]
...,...,...
589,周沛辰,[Apache IoTDB]
590,张凌哲,[Apache IoTDB]
591,李伟豪,[Apache IoTDB]
592,董可伦,[Apache Hudi]


In [24]:
multi_project_employees = employee_projects[employee_projects['project'].apply(lambda x: len(x) > 1)]

multi_project_employees

,reporter,project
68,Benjamin Mahler,"[Aurora, Mesos]"
73,Bhuvaneswaran A,"[Aurora, Mesos]"
74,Bill Farner,"[Aurora, Mesos]"
76,Bjoern Metzdorf,"[Aurora, Mesos]"
97,Chris Lambert,"[Aurora, Mesos]"
133,Dominic Hamon,"[Aurora, Mesos]"
214,Jay Buffington,"[Aurora, Mesos]"
230,Joe Smith,"[Aurora, Mesos]"
286,Lokesh Jain,"[Apache Ozone, Apache Hudi]"
308,Maxim Khutornenko,"[Aurora, Mesos]"


In [30]:
db = mysql.connector.connect(
        host= 'localhost',
        database='sprint_iq',
        user='root',
        password='password'
)

In [31]:

cursor = db.cursor()

In [32]:
# 3. Insert employees into employee table (if not exists)
for _, row in employee_projects.iterrows():
    employee_name = row['reporter']
    try:
        cursor.execute(
            "INSERT IGNORE INTO employee (name) VALUES (%s)",
            (employee_name,)
        )
    except Exception as e:
        print(f"Error inserting employee {employee_name}: {e}")

db.commit()

In [34]:
# 4. Now insert into employee_project table
for _, row in employee_projects.iterrows():
    employee_name = row['reporter']
    projects = row['project']

    # Get employee_id
    cursor.execute(
        "SELECT id FROM employee WHERE name = %s",
        (employee_name,)
    )
    employee_id = cursor.fetchone()[0]

    for project_name in projects:
        # Get project_id (project table already exists)
        cursor.execute(
            "SELECT id FROM projects WHERE name = %s",
            (project_name,)
        )
        result = cursor.fetchone()

        if result:
            project_id = result[0]

            # Insert into employee_project relation
            try:
                cursor.execute(
                    "INSERT IGNORE INTO employee_project (employee_id, project_id) VALUES (%s, %s)",
                    (employee_id, project_id)
                )
            except Exception as e:
                print(f"Error linking employee {employee_name} to project {project_name}: {e}")
        else:
            print(f"Project {project_name} not found!")

db.commit()
db.close()